# 工作簿

## 定义名字

```{note}
定义名称是用于表示单元格，区域，公式或常量值的描述性文本。
```

这意味着它们的定义是非常宽松的。它们可能包含常数，公式，单元格引用，区域或跨不同工作表的多个区域。它们在工作簿全局定义并可以通过 `defined_names` 属性进行访问。

单元格引用或范围必须使用绝对坐标，并且始终包含它们所在的工作表的名称。使用实用程序 `absolute_coordinate()` 和 `quote_sheetname()` 来完成此操作。

定义的名称可以限制为单个工作表，也可以对整个工作簿全局可用。名称在集合中必须是唯一的；新项将用该名称替换现有项。

### 访问全局定义

全局定义存储在工作簿集合中：

```python
defn = wb.defined_names["my_range"]
# the destinations attribute contains a list of ranges in the definitions
dests = defn.destinations # returns a generator of (worksheet title, cell range) tuples

cells = []
for title, coord in dests:
    ws = wb[title]
    cells.append(ws[coord])
```

### 访问工作表定义

分配给特定工作表的定义只能从该工作表中访问:

```python
ws = wb["Sheet"]
defn = ws.defined_names["private_range"]
```

### 创建全局定义

全局定义分配给工作簿集合:

```python
from openpyxl import Workbook
from openpyxl.workbook.defined_name import DefinedName
from openpyxl.utils import quote_sheetname, absolute_coordinate
wb = Workbook()
ws = wb.active
# make sure sheetnames and cell references are quoted correctly
ref =  "{quote_sheetname(ws.title)}!{absolute_coordinate('A1:A5')}"

defn = DefinedName("global_range", attr_text=ref)
wb.defined_names["global_range"] = defn

# key and `name` must be the same, the `.add()` method makes this easy
wb.defined_names.add(new_range)
```

### 创建工作表定义

分配给特定工作表的定义只能从该工作表中访问:

```python
# create a local named range (only valid for a specific sheet)
ws = wb["Sheet"]
ws.title = "My Sheet"
# make sure sheetnames  and cell referencesare quoted correctly
ref = f"{quote_sheetname(ws.title)}!{absolute_coordinate('A6')}"

defn = DefinedName("private_range", attr_text=ref)
ws.defined_names.add(defn)
print(ws.defined_names["private_range"].attr_text)
```

### 动态命名范围

只要相关且可能，`openpyxl` 将尝试将包含单元格范围的名称转换为相关对象。例如，打印区域和打印标题(它们是已定义名称的特殊情况)被映射到工作表中的打印标题和打印区域对象。

但是，可以使用其他已定义的名称或对象(如表)动态定义范围。由于 `openpyxl` 无法解析此类定义，它将跳过该定义并引发警告。如果需要处理这个问题，可以提取已定义名称的范围，并将打印区域设置为适当的单元格范围。

```python
from openpyxl import load_workbook
wb = load_workbook("Example.xlsx")
ws = wb.active
area = ws.defined_names["TestArea"] # Globally defined named ranges can be used too
ws.print_area = area.value          # value is the cell range the defined name currently covers
```